# Decaying isomerization

The decaying isomerization process is the reversible interconversion of two isometric species $S_1$ and $S_2$ combined with the (typically much slower) decay of $S_2$ to a third spcies $S_3$:

$$ S_1 \leftrightarrows S_2 \qquad k_\text{forward} = c_1, k_\text{reverse} = c_2$$
$$ S_2 \to S_3 \qquad k = c_3 $$

As Rathinam [Rathinam 2003] and Gillespie [Gillespie 2008] note, for appropriately chosen parameters, the decaying isomerization problem is the simplest example of a "stiff" problem in chemical kinetics.

In the reversible isomerization process $ S_1 \leftrightarrows S_2$ alone, the random variable describing the number of $S_2$ specimens at equilibrium ($t \to \infty$) is:

$$ P(\# S_2 = x_2) = \text{Binomial}\left(\frac{c1}{c1 + c2}; x_T\right) (x_2) $$

where $x_T$ is the total inital specimens of $S_1$ and $S_2$ combined. (Gillespie 2008).

The relaxation time is defined as the characteristic time for the exponential approach of a perturbed system to its equilibrium. (The characteristic time being the time it takes for the exponential function to be reduced to a fraction $1/e$ of itself.)

In [21]:
import numpy as np
import pandas as pd
from reactionmodel.model import Species, Reaction, Model

In [ ]:
from tests.discover import discover_tests

decaying_isomerization_tests = discover_tests('./', './decaying_isomerization', include_check=True)

In [ ]:
len(decaying_isomerization_tests)

In [24]:
S1 = Species('S1')
S2 = Species('S2')
S3 = Species('S3')

reactions = [
    Reaction([S1], [S2], k='c1'),
    Reaction([S2], [S1], k='c2')
]

m = Model([S1, S2], reactions)

reactions.append(
    Reaction([S2], [S3], k='c3')
)

m_decay = Model([S1, S2, S3], reactions)

p = {'c1': 1, 'c2':1}
p_decay = {'c1': 1.0, 'c2': 2.0, 'c3': 5 * 1e-5} # gillespie 2008 figure 5
#p_decay = {'c1': 1.0, 'c2': 2.0, 'c3': 0}

t_span = [0.0, 50]

initial_dictionary = {'S1': 1200, 'S2': 600, 'S3':0}

ic = m_decay.make_initial_condition(initial_dictionary)

def end_routine(result):
    return m_decay.y_to_dict(result.y)

## Slow-scale stochastic simulation

In [25]:
S12 = Species('S12')
S3  = Species('S3')

reactions = [
    Reaction([S12], [S3], k='c1*c3/(c1+c2)')
]

m_ss = Model([S12, S3], reactions)

ic_ss = m_ss.make_initial_condition({'S12': initial_dictionary['S1']+initial_dictionary['S2'], 'S3': initial_dictionary['S3']})

def sample_y(y_ss, m, p):
    y_dict = {}
    y_ss_dict = m.y_to_dict(y_ss)
    y_dict['S3'] = y_ss_dict['S3']
    y_dict['S2'] = np.random.binomial(y_ss_dict['S12'], p['c1']/(p['c1'] + p['c2']))
    y_dict['S1'] = y_ss_dict['S12'] - y_dict['S2']
    return y_dict

In [ ]:
from hybrid.gillespie import GillespieSimulator

g_ss = GillespieSimulator(
    m_ss.get_k(parameters=p_decay, jit=True),
    m_ss.stoichiometry(),
    m_ss.kinetic_order(),
)

In [ ]:
def ss_end_routine(result):
    return sample_y(result.y, m_ss, p_decay)

ss_df = pd.DataFrame(g_ss.run_simulations(1000, t_span, ic_ss, np.random.default_rng(), end_routine=ss_end_routine))
ss_df

In [ ]:
ss_df['S3'].hist()

In [29]:
from hybrid.parse import PreconfiguredSimulatorLoader

loader = PreconfiguredSimulatorLoader.load_preconfigured('hybrid_n')

In [ ]:
hc_s = loader.make_simulator(
    m_decay.get_k(parameters=p_decay, jit=True),
    m_decay.stoichiometry(),
    m_decay.kinetic_order(),
)

In [31]:
#pd.DataFrame(hc_s.run_simulations(1, t_span, ic, np.random.default_rng(), end_routine=end_routine))

## [Oct 2024] Hybrid rounding questions

In [32]:
for t in decaying_isomerization_tests:
    _, name, parse_results, _ = t
    if name == 'least_stiff_shortest_hybrid_cle_em_nonstoich':
        target_test = parse_results
        break

In [ ]:
target_test

In [ ]:
model = target_test.model
s_factory = target_test.simulator_config
s = s_factory.make_simulator_from_model(model, parameters=target_test.parameters)

In [ ]:
model.make_initial_condition

In [36]:
r = s.simulate(parse_results.t.t_span, model.make_initial_condition(parse_results.initial_condition), rng=np.random.default_rng())

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=13)
ax = r.plot(m.legend())
ax.plot(r.t_history, r.y_history.sum(axis=0))
ax.set_ylim(0, 1850)
ax.set_yticks(np.arange(0, 2000, 600))
ax.axhline(1800, c='r', alpha=0.2)
ax.set_ylabel('specimens')

plt.savefig('decaying_rounding.png', dpi=300)

In [ ]:
r.y.sum()

## Hybrid

In [ ]:
from hybrid.hybrid import HybridSimulator, FixedThresholdPartitioner

h_s = HybridSimulator(
    m_decay.get_k(parameters=p_decay, jit=True),
    m_decay.stoichiometry(),
    m_decay.kinetic_order(),
    FixedThresholdPartitioner(100.0),
    fast_scale='langevin',
    approximate_rtot=True,
    contrived_no_reaction_rate=20.,
    euler_maruyama_timestep=0.01,
    round='randomly',
)

standard_gillespie_s = GillespieSimulator(
    m_decay.get_k(parameters=p_decay, jit=True),
    m_decay.stoichiometry(),
    m_decay.kinetic_order(),    
)

In [ ]:
result = h_s.simulate([0, 50], ic, np.random.default_rng())
result.plot(m_decay.legend())

In [ ]:
g_result = standard_gillespie_s.simulate([0, 50], ic, np.random.default_rng())
g_result.plot(m_decay.legend())

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import kstest

cle_distribution = result.y_history[0, :]
gillespie_dist = g_result.y_history[0, :]

with_ss = False

labels = ['gillespie', 'langevin']
dists = [gillespie_dist, cle_distribution]
ss_distribution = 1800-np.random.binomial(1800, p_decay['c1']/(p_decay['c1'] + p_decay['c2']), size=20000)

if with_ss:
    labels.append('ss')
    dists.append(ss_distribution)
plt.hist(dists, label=labels, density=True, bins=20)
plt.legend(labels)

print(kstest(cle_distribution, gillespie_dist))
kstest(gillespie_dist, ss_distribution)

In [ ]:
test_one = 1800-np.random.binomial(1800, p_decay['c1']/(p_decay['c1'] + p_decay['c2']), size=20000)
test_two = 1800-np.random.binomial(1800, p_decay['c1']/(p_decay['c1'] + p_decay['c2']), size=20000)

plt.hist([test_one, test_two], label=['t1', 't2'], density=True, bins=20)

kstest(test_one, test_two)

In [ ]:
len(cle_distribution), np.mean(cle_distribution), np.var(cle_distribution)

In [ ]:
len(gillespie_dist), np.mean(gillespie_dist), np.var(gillespie_dist)

## Tau leaping

In [ ]:
m_decay.all_reactions

In [ ]:
from hybrid.tau import TauLeapSimulator

equilibrium_mask = np.array([True, True, False])

t_s = TauLeapSimulator(
    m_decay.get_k(parameters=p_decay, jit=True),
    m_decay.stoichiometry(),
    m_decay.kinetic_order(),
    method='implicit',
    epsilon=0.01,
    equilibrium_mask=equilibrium_mask
)

In [7]:
t_span = [0, 20000]

In [ ]:
rng = np.random.default_rng()
result = t_s.simulate(t_span, ic, rng, history_length=1e8)
np.diff(result.y_history[2, :])

In [ ]:
np.diff(result.y_history[2, :])

In [ ]:
result.plot(m_decay.legend())

In [ ]:
sum(result.y_history[:, -1])

In [ ]:
import matplotlib.pyplot as plt
taus = [result.t_history[i+1] - result.t_history[i] for i in range(len(result.t_history)-1)]
plt.hist(np.array(taus))

In [24]:
tau_df = pd.DataFrame(t_s.run_simulations(120, t_span, ic, np.random.default_rng(), end_routine=end_routine))

In [ ]:
tau_df['S3'].hist()

In [ ]:
from scipy.stats import kstest

kstest(tau_df['S3'], ss_df['S3'])

In [27]:
result = t_s.simulate(t_span, ic, np.random.default_rng())

In [ ]:
result.plot(m_decay.legend())
result.status_counter

# Immigration + linear death process

In [ ]:
from reactionmodel.model import Species, Reaction, Model
from hybrid.gillespie import GillespieSimulator

S1 = Species('S1')

reactions = [
    Reaction([S1], [], k='r'),
    Reaction([], [S1], k='p')
]

m = Model([S1], reactions)

p = {'r': 1, 'p':100}

s = GillespieSimulator(
    m.get_k(parameters=p, jit=True),
    m.stoichiometry(),
    m.kinetic_order(),
)

In [ ]:
import numpy as np
result = s.simulate([0.0, 28.0], m.make_initial_condition({'S1': 100}), np.random.default_rng())
ax = result.plot(['Y_s'])
ax.axhline(100, color='r', linestyle='dashed')

In [31]:
import matplotlib.pyplot as plt
#plt.hist(result.y_history)

In [ ]:
import pandas as pd
r = pd.DataFrame(data={'S': np.squeeze(result.y_history.T)})
r

In [ ]:
import scipy.stats

ax = plt.subplot()
x = np.arange(r.min().iloc[0], r.max().iloc[0])

ax.plot([x[0] for x in r.value_counts(normalize=True).sort_index().index.to_numpy()], r.value_counts(normalize=True).sort_index().to_numpy())
#ax = r.value_counts(normalize=True).sort_index().plot()

rv = scipy.stats.poisson(p['p']/p['c1'])
ax.plot(x, rv.pmf(x))

# Test discovery

In [ ]:
import tests.discover as discover

discover.discover_tests('./', include_check=True)

# Simple experiment idea:

Let's say you have a latent reservoir of size $n$ with variance $v$. As a function of the variance in the reservoir size, how much variance is there in the time until you hit exponential growth? How much variation is there in the re-seeded latent reservoir after a phase of $R_0$ that lasts for $t$ days?

=> how does this translate to your conclusions about probability of mutant emergence given a set standard?
=> how 